# 서울시 기온 데이터를 이용해서 기상청 예보와 얼마나 차이나는지 비교하자

### 사용 데이터: 서울시 평균기온, 최저기온, 최고기온 (2000년 10월 17일~ 2021년 10월 17일)

In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import mglearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression # 선형회귀
from sklearn.neighbors import KNeighborsRegressor # KNN 회귀
from sklearn.tree import DecisionTreeRegressor # 결정트리회귀
from sklearn.ensemble import RandomForestRegressor # 랜덤포레스트 회귀
from sklearn.model_selection import cross_val_score # 교차검증
from sklearn.model_selection import KFold # KFold 교차검증

In [5]:
temp = pd.read_csv('./data/서울기온_20001017_20211017.csv', encoding='cp949')

In [6]:
temp

,날짜,지점,평균기온(℃),최저기온(℃),최고기온(℃)
0,2000-10-17,108,12.4,8.5,15.1
1,2000-10-18,108,14.5,9.9,19.8
2,2000-10-19,108,14.2,10.1,19.2
3,2000-10-20,108,15.9,11.8,20.9
4,2000-10-21,108,14.7,9.0,20.8
...,...,...,...,...,...
7666,2021-10-13,108,19.6,14.9,25.2
7667,2021-10-14,108,19.7,17.4,23.9
7668,2021-10-15,108,19.0,17.0,21.4
7669,2021-10-16,108,10.7,4.2,18.1


In [7]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7671 entries, 0 to 7670
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   날짜       7671 non-null   object 
 1   지점       7671 non-null   int64  
 2   평균기온(℃)  7671 non-null   float64
 3   최저기온(℃)  7671 non-null   float64
 4   최고기온(℃)  7670 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 299.8+ KB


### 데이터 전처리 목표
* 최고기온 컬럼에 결측치가 하나 있으니 그 행을 제외하거나 결측치를 채워주자
* 날짜 Dtype이 object로 되어있으니 숫자 데이터로 나눠주자
* 지점은 서울 하나 뿐이니까 무의미한 데이터이므로 제외하자

In [8]:
temp = temp.dropna() # 결측치 제외

In [9]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7670 entries, 0 to 7670
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   날짜       7670 non-null   object 
 1   지점       7670 non-null   int64  
 2   평균기온(℃)  7670 non-null   float64
 3   최저기온(℃)  7670 non-null   float64
 4   최고기온(℃)  7670 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 359.5+ KB


In [10]:
temp['지점'].unique()

array([108], dtype=int64)

In [12]:
temp.drop(columns = ['지점'], inplace=True) # 지점 col 제외

C:\Users\Administrator\anaconda3\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
temp

,날짜,평균기온(℃),최저기온(℃),최고기온(℃)
0,2000-10-17,12.4,8.5,15.1
1,2000-10-18,14.5,9.9,19.8
2,2000-10-19,14.2,10.1,19.2
3,2000-10-20,15.9,11.8,20.9
4,2000-10-21,14.7,9.0,20.8
...,...,...,...,...
7666,2021-10-13,19.6,14.9,25.2
7667,2021-10-14,19.7,17.4,23.9
7668,2021-10-15,19.0,17.0,21.4
7669,2021-10-16,10.7,4.2,18.1


In [14]:
temp['날짜'].dtype

dtype('O')

In [15]:
temp['날짜'] = pd.to_datetime(temp['날짜'])

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_15804/2400803615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['날짜'] = pd.to_datetime(temp['날짜'])


In [18]:
temp['년'] = temp['날짜'].dt.year
temp['월'] = temp['날짜'].dt.month
temp['일'] = temp['날짜'].dt.day

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_15804/4107011839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['년'] = temp['날짜'].dt.year
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_15804/4107011839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['월'] = temp['날짜'].dt.month
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_15804/4107011839.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [19]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7670 entries, 0 to 7670
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   날짜       7670 non-null   datetime64[ns]
 1   평균기온(℃)  7670 non-null   float64       
 2   최저기온(℃)  7670 non-null   float64       
 3   최고기온(℃)  7670 non-null   float64       
 4   년        7670 non-null   int64         
 5   월        7670 non-null   int64         
 6   일        7670 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 479.4 KB


### 데이터를 train과 test용으로 나누기

In [20]:
X = temp.loc[:, ['년', '월', '일', '최저기온(℃)', '최고기온(℃)']]
y = temp['평균기온(℃)']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [22]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5752, 5), (1918, 5), (5752,), (1918,))

### 머신러닝 모델 생성, 학습, score

In [23]:
model_linear = LinearRegression()
model_knn = KNeighborsRegressor()
model_tree = DecisionTreeRegressor()
model_random = RandomForestRegressor()

In [24]:
model_linear.fit(X_train, y_train)
model_knn.fit(X_train, y_train)
model_tree.fit(X_train, y_train)
model_random.fit(X_train, y_train)

RandomForestRegressor()

In [25]:
model_linear.score(X_test, y_test)

0.9975943606262837

In [26]:
model_knn.score(X_test, y_test)

0.995176652899365

In [27]:
model_tree.score(X_test, y_test)

0.9950184055257912

In [28]:
model_random.score(X_test, y_test)

0.9972830567179243

In [33]:
scores = cross_val_score(model_linear, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_linear,scores))
scores = cross_val_score(model_knn, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_knn,scores))
scores = cross_val_score(model_tree, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_tree,scores))
scores = cross_val_score(model_random, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_random,scores))

LinearRegression() 교차 검증 점수 : [0.99800014 0.99768311 0.99719069 0.99735508 0.99766354]
KNeighborsRegressor() 교차 검증 점수 : [0.99368619 0.98899898 0.99055629 0.99254086 0.99154082]
DecisionTreeRegressor() 교차 검증 점수 : [0.99358306 0.99437374 0.99363698 0.99408343 0.99339403]
RandomForestRegressor() 교차 검증 점수 : [0.99708814 0.99733428 0.99663923 0.9966298  0.9970805 ]


In [36]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("{} 교차 검증 점수 : {}".format(model_linear,cross_val_score(model_linear, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_knn,cross_val_score(model_knn, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_tree,cross_val_score(model_tree, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_random,cross_val_score(model_random, X_test, y_test, cv=kfold)))

LinearRegression() 교차 검증 점수 : [0.99758947 0.99761331 0.99753586]
KNeighborsRegressor() 교차 검증 점수 : [0.99269471 0.99135975 0.99007244]
DecisionTreeRegressor() 교차 검증 점수 : [0.99387201 0.99355696 0.99384843]
RandomForestRegressor() 교차 검증 점수 : [0.99713987 0.99706787 0.99643261]


## 결과
* 네 가지 모델 모두 매우 높은 정확도를 기록
* 예측할 데이터가 단순해서 그런 것 같다.
* 평균기온 예측은 최저기온과 최고기온을 더해서 나누면 매우 가까운 값을 얻을 수 있다는 점도 영향이 있는 것 같다.

### 조건을 바꿔가며 해보자
* 최저기온, 최고기온을 다 주지 말고, 한 두개씩 빼면서 해보자

## 최저기온 빼기

### 데이터를 train과 test용으로 나누기

In [37]:
X = temp.loc[:, ['년', '월', '일', '최고기온(℃)']]
y = temp['평균기온(℃)']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [38]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5752, 4), (1918, 4), (5752,), (1918,))

### 머신러닝 모델 생성, 학습, score

In [39]:
model_linear = LinearRegression()
model_knn = KNeighborsRegressor()
model_tree = DecisionTreeRegressor()
model_random = RandomForestRegressor()

model_linear.fit(X_train, y_train)
model_knn.fit(X_train, y_train)
model_tree.fit(X_train, y_train)
model_random.fit(X_train, y_train)

RandomForestRegressor()

In [40]:
print(model_linear.score(X_test, y_test))
print(model_knn.score(X_test, y_test))
print(model_tree.score(X_test, y_test))
print(model_random.score(X_test, y_test))

0.9811857322828986
0.981889206100481
0.9771422627580904
0.9876709363972933


In [41]:
scores = cross_val_score(model_linear, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_linear,scores))
scores = cross_val_score(model_knn, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_knn,scores))
scores = cross_val_score(model_tree, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_tree,scores))
scores = cross_val_score(model_random, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_random,scores))

LinearRegression() 교차 검증 점수 : [0.98164385 0.98318734 0.97830378 0.98181618 0.98040164]
KNeighborsRegressor() 교차 검증 점수 : [0.97862222 0.97419448 0.97218335 0.97585135 0.97304807]
DecisionTreeRegressor() 교차 검증 점수 : [0.97517088 0.97594806 0.97443405 0.97375869 0.97535933]
RandomForestRegressor() 교차 검증 점수 : [0.98640503 0.98622491 0.98532276 0.98451648 0.98712423]


In [42]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("{} 교차 검증 점수 : {}".format(model_linear,cross_val_score(model_linear, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_knn,cross_val_score(model_knn, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_tree,cross_val_score(model_tree, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_random,cross_val_score(model_random, X_test, y_test, cv=kfold)))

LinearRegression() 교차 검증 점수 : [0.98091711 0.98187307 0.98086025]
KNeighborsRegressor() 교차 검증 점수 : [0.97684783 0.97545444 0.9724428 ]
DecisionTreeRegressor() 교차 검증 점수 : [0.97484931 0.97699112 0.97465834]
RandomForestRegressor() 교차 검증 점수 : [0.98521759 0.9864858  0.98476462]


## 최고기온 빼기

### 데이터를 train과 test용으로 나누기

In [54]:
X = temp.loc[:, ['월', '일', '최저기온(℃)']]
y = temp['평균기온(℃)']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5752, 3), (1918, 3), (5752,), (1918,))

### 머신러닝 모델 생성, 학습, score

In [55]:
model_linear = LinearRegression()
model_knn = KNeighborsRegressor()
model_tree = DecisionTreeRegressor()
model_random = RandomForestRegressor()

model_linear.fit(X_train, y_train)
model_knn.fit(X_train, y_train)
model_tree.fit(X_train, y_train)
model_random.fit(X_train, y_train)

RandomForestRegressor()

In [56]:
print(model_linear.score(X_test, y_test))
print(model_knn.score(X_test, y_test))
print(model_tree.score(X_test, y_test))
print(model_random.score(X_test, y_test))

0.9794700129511802
0.9788714351424813
0.9655193414686226
0.9788139001877763


In [58]:
scores = cross_val_score(model_linear, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_linear,scores))
scores = cross_val_score(model_knn, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_knn,scores))
scores = cross_val_score(model_tree, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_tree,scores))
scores = cross_val_score(model_random, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_random,scores))

LinearRegression() 교차 검증 점수 : [0.98011327 0.98125317 0.97520023 0.98127495 0.97924664]
KNeighborsRegressor() 교차 검증 점수 : [0.97800825 0.97545791 0.97237736 0.97793173 0.9775423 ]
DecisionTreeRegressor() 교차 검증 점수 : [0.9654769  0.96504479 0.95582851 0.96873563 0.96365252]
RandomForestRegressor() 교차 검증 점수 : [0.979225   0.9795927  0.97570484 0.98020713 0.97786617]


In [59]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("{} 교차 검증 점수 : {}".format(model_linear,cross_val_score(model_linear, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_knn,cross_val_score(model_knn, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_tree,cross_val_score(model_tree, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_random,cross_val_score(model_random, X_test, y_test, cv=kfold)))

LinearRegression() 교차 검증 점수 : [0.97796824 0.98123859 0.97934084]
KNeighborsRegressor() 교차 검증 점수 : [0.9760752  0.97681841 0.97549467]
DecisionTreeRegressor() 교차 검증 점수 : [0.95968955 0.96472736 0.96348722]
RandomForestRegressor() 교차 검증 점수 : [0.97696535 0.97880993 0.977969  ]


## 년, 월, 일만 주기

### 데이터를 train과 test용으로 나누기

In [43]:
X = temp.loc[:, ['년', '월', '일']]
y = temp['평균기온(℃)']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [44]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5752, 3), (1918, 3), (5752,), (1918,))

### 머신러닝 모델 생성, 학습, score

In [45]:
model_linear = LinearRegression()
model_knn = KNeighborsRegressor()
model_tree = DecisionTreeRegressor()
model_random = RandomForestRegressor()

model_linear.fit(X_train, y_train)
model_knn.fit(X_train, y_train)
model_tree.fit(X_train, y_train)
model_random.fit(X_train, y_train)

RandomForestRegressor()

In [46]:
print(model_linear.score(X_test, y_test))
print(model_knn.score(X_test, y_test))
print(model_tree.score(X_test, y_test))
print(model_random.score(X_test, y_test))

0.04283399309423075
0.9011047108556656
0.9126617194159998
0.9422052870282864


In [47]:
scores = cross_val_score(model_linear, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_linear,scores))
scores = cross_val_score(model_knn, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_knn,scores))
scores = cross_val_score(model_tree, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_tree,scores))
scores = cross_val_score(model_random, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_random,scores))

LinearRegression() 교차 검증 점수 : [0.03285193 0.07060361 0.05712226 0.01347061 0.0381296 ]
KNeighborsRegressor() 교차 검증 점수 : [0.83688246 0.82109265 0.83983981 0.81602294 0.82545688]
DecisionTreeRegressor() 교차 검증 점수 : [0.84031831 0.850267   0.82368783 0.85187857 0.88311575]
RandomForestRegressor() 교차 검증 점수 : [0.91221685 0.9005666  0.89024243 0.91482097 0.92323916]


In [48]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("{} 교차 검증 점수 : {}".format(model_linear,cross_val_score(model_linear, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_knn,cross_val_score(model_knn, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_tree,cross_val_score(model_tree, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_random,cross_val_score(model_random, X_test, y_test, cv=kfold)))

LinearRegression() 교차 검증 점수 : [0.0217345  0.0344994  0.06357378]
KNeighborsRegressor() 교차 검증 점수 : [0.82319608 0.82644611 0.8083314 ]
DecisionTreeRegressor() 교차 검증 점수 : [0.85496253 0.83685057 0.83896256]
RandomForestRegressor() 교차 검증 점수 : [0.9063645  0.89001553 0.90069123]


## 월, 일만 주기

### 데이터를 train과 test용으로 나누기

In [89]:
X = temp.loc[:, ['월', '일']]
y = temp['평균기온(℃)']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5752, 2), (1918, 2), (5752,), (1918,))

### 머신러닝 모델 생성, 학습, score

In [90]:
model_linear = LinearRegression()
model_knn = KNeighborsRegressor()
model_tree = DecisionTreeRegressor()
model_random = RandomForestRegressor()

model_linear.fit(X_train, y_train)
model_knn.fit(X_train, y_train)
model_tree.fit(X_train, y_train)
model_random.fit(X_train, y_train)

RandomForestRegressor()

In [91]:
print(model_linear.score(X_test, y_test))
print(model_knn.score(X_test, y_test))
print(model_tree.score(X_test, y_test))
print(model_random.score(X_test, y_test))

0.04212199113859927
0.8824110193979088
0.8970834537455534
0.8970090630681526


In [92]:
scores = cross_val_score(model_linear, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_linear,scores))
scores = cross_val_score(model_knn, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_knn,scores))
scores = cross_val_score(model_tree, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_tree,scores))
scores = cross_val_score(model_random, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_random,scores))

LinearRegression() 교차 검증 점수 : [0.03530681 0.06691995 0.0605549  0.0131162  0.03541651]
KNeighborsRegressor() 교차 검증 점수 : [0.86419576 0.86871295 0.84825498 0.88398667 0.88172962]
DecisionTreeRegressor() 교차 검증 점수 : [0.88302465 0.86626415 0.86110334 0.88868069 0.89727183]
RandomForestRegressor() 교차 검증 점수 : [0.88593416 0.874108   0.86553105 0.89163488 0.90072816]


In [93]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("{} 교차 검증 점수 : {}".format(model_linear,cross_val_score(model_linear, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_knn,cross_val_score(model_knn, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_tree,cross_val_score(model_tree, X_test, y_test, cv=kfold)))
print("{} 교차 검증 점수 : {}".format(model_random,cross_val_score(model_random, X_test, y_test, cv=kfold)))

LinearRegression() 교차 검증 점수 : [0.02931349 0.03244428 0.06164877]
KNeighborsRegressor() 교차 검증 점수 : [0.87339462 0.86011978 0.87714137]
DecisionTreeRegressor() 교차 검증 점수 : [0.88120649 0.86386363 0.87841809]
RandomForestRegressor() 교차 검증 점수 : [0.88805602 0.87452104 0.88342333]


## 결과
* 데이터 열을 빼면 뺄수록 정확도가 떨어진다.
* 선형회귀는 최저기온이나 최고기온 둘 중 어느 쪽을 제거해도 높은 정확도를 보였다.
* 그러나 둘 다 제거할 경우 정확도가 매우 낮았다.
* 반면에 다른 모델들은 시간 데이터만 제공받은 경우에도 꽤나 높은 정확도를 보였다.
* 평균기온 예측은 최저기온과 최고기온을 더해서 나누면 매우 가까운 값을 얻을 수 있다는 점이 확실히 영향이 있는 것 같다.
* 다른 년도의 동일 날짜의 기온으로 예측하면, 대충 90퍼 정도는 맞는다는 소리

### 제일 궁금했던, 날짜만 주어진 상태에서 예측한 결과는 얼마나 정확할까?

In [95]:
target_date = pd.DataFrame()

In [96]:
target_date['월'] = [10, 10, 10]
target_date['일'] = [18, 19, 20]

In [98]:
target_date

,월,일
0,10,18
1,10,19
2,10,20


In [101]:
pred_linear = model_linear.predict(target_date)
pred_knn = model_knn.predict(target_date)
pred_tree = model_tree.predict(target_date)
pred_random = model_random.predict(target_date)

In [102]:
print(pred_linear)
print(pred_knn)
print(pred_tree)
print(pred_random)

[15.70620101 15.71802526 15.72984951]
[14.54 14.36 16.1 ]
[14.75       15.03571429 15.05      ]
[14.59457104 15.06770332 14.90668889]


### 실제 데이터는?

* 18일: 평균기온:9.6℃
* 19일: 평균기온:10.9℃

### 기상청은 어떻게 예측했나?

* 18일: 평균기온: 9℃
* 19일: 평균기온: 9.5℃
* 20일: 평균기온: 10℃

### 다른 년도 데이터는?
* 2020년
* 18일: 평균기온:13.6℃
* 19일: 평균기온:15.1℃
* 20일: 평균기온:15.2℃

## 최종 결과

* 머신 러닝 모델은 기온이 다른 년도와 비슷하다면, 월과 일 데이터만 가지고도 매우 비슷한 결과를 도출할 수 있었다.
* 근데 이건 머신 러닝 없어도 할 수 있으니까, 별 의미는 없는 것 같다.
* 2021년 올해처럼 갑자기 추워질 경우 머신러닝 모델은 정확하게 예측하지 못했다.

### 배운 점
* 태블로를 이용해서 시각화 하는 방법을 익히고 실습할 수 있었다.
* 예측하려는 데이터와 가지고 있는 데이터가 지나치게 관계가 깊으면 사실상 머신 러닝을 이용하는 의미가 없다.
    + 평균 기온과 최고 기온, 최저 기온은 하나를 알면 사실상 나머지를 추측하기 너무나 쉬워진다.
* 과거에 쌓인 데이터대로만 예상하는 건 머신러닝 모델이 잘 한다.
    + 그러나 이건 머신 러닝 없어도 가능하니까 어떤 문제가 머신 러닝으로 풀어야 하는 문제인지 잘 정의하는 것이 중요하다는 걸 배웠다.